# <a href="http://www.datascience-paris-saclay.fr">Paris Saclay Center for Data Science</a>
# <a href=https://ramp.r0h.eu/problems/air_passengers>RAMP</a> on predicting the number of air passengers

<i> Balázs Kégl (LAL/CNRS), Alex Gramfort (LTCI/Telecom ParisTech), Djalel Benbouzid (UPMC), Mehdi Cherti (LAL/CNRS) </i>

## Introduction
The data set was donated to us by an unnamed company handling flight ticket reservations. The data is thin, it contains
<ul>
<li> the date of departure
<li> the departure airport
<li> the arrival airport
<li> the mean and standard deviation of the number of weeks of the reservations made before the departure date
<li> a field called <code>log_PAX</code> which is related to the number of passengers (the actual number were changed for privacy reasons)
</ul>

The goal is to predict the <code>log_PAX</code> column. The prediction quality is measured by RMSE. 

The data is obviously limited, but since data and location informations are available, it can be joined to external data sets. <b>The challenge in this RAMP is to find good data that can be correlated to flight traffic</b>.

To do :
- mean encoding 
- regrouper variables catégoriques
- ajouter nouvelles données : distance aéroports (trouver fichier avec données)
- faire analyses descriptives

In [53]:
%matplotlib inline
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

## Fetch the data and load it in pandas

First we load `problem.py` that parameterizes the challenge. It contains some objects taken off the shelf from `ramp-workflow` (e.g., `Predictions` type, scores, and data reader). 

In [54]:
problem = imp.load_source('', 'problem.py')

`get_train_data` loads the training data and returns an `pandas` object (input) and a `np.array` object (output).

In [55]:
X_df, y_array = problem.get_train_data()
# the y array contains the log-like number of passengers on each flight 

In [56]:
print(min(X_df['DateOfDeparture']))
print(max(X_df['DateOfDeparture']))

2011-09-01
2013-03-05


In [57]:
X_df['log_PAX'] = y_array
cols = X_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
X_df = X_df[cols]
X_encoded = X_df
X_encoded.head()

,log_PAX,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
0,12.331296,2012-06-19,ORD,DFW,12.875000,9.812647
1,10.775182,2012-09-10,LAS,DEN,14.285714,9.466734
2,11.083177,2012-10-05,DEN,LAX,10.863636,9.035883
3,11.169268,2011-10-09,ATL,ORD,11.480000,7.990202
4,11.269364,2012-02-21,DEN,SFO,11.450000,9.517159


In [58]:
ext_data = pd.read_csv('submissions/starting_kit/external_data.csv', sep=',')
ext_data.head()

,Departure,Arrival,Distance,DateOfDeparture,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Number_hab,Revenue,Oil_price,dep_encod,ar_encod
0,LAS,ORD,2434,2011-09-01,34,28,22,23,22,19,84,70,56,1015,1013,1011,16,15,11,26,9,37.0,0,1,NaN,197,2712920,52273,88.93,10.79,11.26
1,LAS,ORD,2434,2011-09-02,33,29,24,22,21,19,79,64,49,1013,1012,1011,16,15,11,27,12,37.0,0,4,NaN,215,2712920,52273,86.57,10.79,11.26
2,LAS,ORD,2434,2011-09-03,32,26,21,22,20,18,93,70,46,1013,1012,1011,16,15,8,32,10,39.0,0.51,6,Rain-Thunderstorm,270,2712920,52273,93.79,10.79,11.26
3,LAS,ORD,2434,2011-09-04,23,19,14,19,13,9,93,70,47,1017,1013,1010,16,16,16,35,19,45.0,0,5,NaN,312,2712920,52273,93.79,10.79,11.26
4,LAS,ORD,2434,2011-09-05,18,15,11,10,7,6,77,64,51,1021,1020,1017,16,16,16,42,24,56.0,0,3,NaN,5,2712920,52273,83.52,10.79,11.26


In [59]:
X_encoded = X_df
external_data = ext_data[['DateOfDeparture', 'Departure', 'Arrival', 'Distance',
                      'dep_encod', 'ar_encod', 'Revenue'
                      ]]
X_encoded = pd.merge(
            X_encoded, external_data, how='left',
            left_on=['DateOfDeparture', 'Arrival','Departure'],
            right_on=['DateOfDeparture', 'Arrival','Departure'],
            sort=False)
X_encoded.head()

,log_PAX,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,Distance,dep_encod,ar_encod,Revenue
0,12.331296,2012-06-19,ORD,DFW,12.875000,9.812647,1292,11.29,11.09,45616
1,10.775182,2012-09-10,LAS,DEN,14.285714,9.466734,1010,10.79,10.68,52262
2,11.083177,2012-10-05,DEN,LAX,10.863636,9.035883,1387,10.68,11.43,57739
3,11.169268,2011-10-09,ATL,ORD,11.480000,7.990202,977,10.99,11.26,52273
4,11.269364,2012-02-21,DEN,SFO,11.450000,9.517159,1554,10.68,11.23,57739


In [60]:
X_encoded.drop(['std_wtd'], axis=1, inplace=True)

In [61]:
X_encoded['Distance_int'] = pd.qcut(X_encoded['Distance'], 4)
X_encoded['Distance_int'].values

[(977.0, 1428.0], (977.0, 1428.0], (977.0, 1428.0], (296.999, 977.0], (1428.0, 2234.0], ..., (296.999, 977.0], (977.0, 1428.0], (296.999, 977.0], (977.0, 1428.0], (296.999, 977.0]]
Length: 8902
Categories (4, interval[float64]): [(296.999, 977.0] < (977.0, 1428.0] < (1428.0, 2234.0] < (2234.0, 4346.0]]

In [52]:
X_encoded['Distance'].min()

297

In [51]:
print(X_encoded.loc[[4384]])

        log_PAX DateOfDeparture  WeeksToDeparture  Distance  dep_encod  \
4384  13.401985      2012-08-02         17.162162      3979      11.31   

      ar_encod  Revenue  d_ATL  d_BOS  d_CLT  d_DEN  d_DFW  d_DTW  d_EWR  \
4384     11.43    57739      0      0      0      0      0      0      0   

      d_IAH  d_JFK  d_LAS  d_LAX  d_LGA  d_MCO  d_MIA  d_MSP  d_ORD  d_PHL  \
4384      0      1      0      0      0      0      0      0      0      0   

      d_PHX  d_SEA  d_SFO  a_ATL  a_BOS  a_CLT  a_DEN  a_DFW  a_DTW  a_EWR  \
4384      0      0      0      0      0      0      0      0      0      0   

      a_IAH  a_JFK  a_LAS  a_LAX  a_LGA  a_MCO  a_MIA  a_MSP  a_ORD  a_PHL  \
4384      0      0      0      1      0      0      0      0      0      0   

      a_PHX  a_SEA  a_SFO  year  month  day  weekday  week  n_days  y_2011  \
4384      0      0      0  2012      8    2        3    31   15554       0   

      y_2012  y_2013  m_1  m_2  m_3  m_4  m_5  m_6  m_7  m_8  m_9  m_1

In [35]:
from sklearn.preprocessing import StandardScaler

X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))
X_encoded = X_encoded.drop('Departure', axis=1)
X_encoded = X_encoded.drop('Arrival', axis=1)

X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])

X_encoded['year']    = X_encoded['DateOfDeparture'].dt.year
X_encoded['month']   = X_encoded['DateOfDeparture'].dt.month
X_encoded['day']     = X_encoded['DateOfDeparture'].dt.day
X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
X_encoded['week']    = X_encoded['DateOfDeparture'].dt.week
X_encoded['n_days']  = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))




In [ ]:
X_encoded[['day', 'log_PAX']].groupby(['day']).mean().sort_values(by='log_PAX', ascending=False)

In [ ]:
X_encoded[['year', 'log_PAX']].groupby(['year']).mean().sort_values(by='log_PAX', ascending=False)

In [ ]:
X_encoded[['month', 'log_PAX']].groupby(['month']).mean().sort_values(by='log_PAX', ascending=False)

In [ ]:
X_encoded[['week', 'log_PAX']].groupby(['week']).mean().sort_values(by='log_PAX', ascending=False)

In [ ]:
X_encoded[['weekday', 'log_PAX']].groupby(['weekday']).mean().sort_values(by='log_PAX', ascending=False)

In [ ]:
X_encoded.drop(['year', 'month', 'day', 'weekday', 'week', 'n_days','DateOfDeparture'], axis=1, inplace=True)

In [ ]:
X_encoded['Events'].fillna(0, inplace=True)
X_encoded['Events'] = X_encoded['Events'].replace(['Rain','Fog'], 1)
X_encoded['Events'] = X_encoded['Events'].replace(['Rain-Thunderstorm','Fog-Rain-Thunderstorm', 
                                                         'Rain-Snow','Snow','Fog-Rain','Thunderstorm','Fog-Snow', 
                                                         'Fog-Rain-Snow','Fog-Rain-Snow-Thunderstorm', 
                                                         'Rain-Snow-Thunderstorm','Rain-Hail-Thunderstorm', 
                                                         'Fog-Rain-Hail-Thunderstorm', 
                                                         'Rain-Thunderstorm-Tornado'], 2)

In [ ]:
X_encoded['Precipitationmm'] = X_encoded['Precipitationmm'].replace('T', np.nan)
X_encoded['Precipitationmm'] = pd.to_numeric(X_encoded['Precipitationmm'])
X_encoded['Precipitationmm'] = X_encoded['Precipitationmm'].fillna(X_encoded['Precipitationmm'].mean())

In [ ]:
X_encoded['Precipitationmm*CloudCover'] = X_encoded.Precipitationmm * X_encoded.CloudCover
X_encoded['Temperature*Humidity'] = X_encoded['Mean TemperatureC'] * X_encoded['Mean Humidity']

In [8]:
X_encoded['Weeks_to_dep_int'] = pd.qcut(X_encoded['WeeksToDeparture'], 4)
X_encoded.loc[X_encoded['WeeksToDeparture'] <= 9.524, 'WeeksToDeparture'] = 0
X_encoded.loc[(X_encoded['WeeksToDeparture'] > 9.524) & (X_encoded['WeeksToDeparture'] <= 11.3), 'WeeksToDeparture'] = 1
X_encoded.loc[(X_encoded['WeeksToDeparture'] > 11.3) & (X_encoded['WeeksToDeparture'] <= 13.24), 'WeeksToDeparture'] = 2
X_encoded.loc[ X_encoded['WeeksToDeparture'] > 13.24, 'WeeksToDeparture'] = 3
X_encoded['WeeksToDeparture'] = X_encoded['WeeksToDeparture'].astype(int)
X_encoded.drop(['Weeks_to_dep_int'], axis=1, inplace=True)

In [10]:
X_encoded[['WeeksToDeparture', 'log_PAX']].groupby(['WeeksToDeparture']).mean().sort_values(by='log_PAX', ascending=False)

,log_PAX
WeeksToDeparture,
3,11.213970
2,11.010436
1,10.934104
0,10.838703


In [ ]:
X_encoded.loc[:, 'Precipitationmm*CloudCover' : 'Temperature*Humidity'] = StandardScaler().fit_transform(
                                                            X_encoded.loc[:, 'Precipitationmm*CloudCover' : 'Temperature*Humidity'])

X_encoded.loc[:, 'Distance' : 'Max Wind SpeedKm/h'] = StandardScaler().fit_transform(
                                                            X_encoded.loc[:, 'Distance' : 'Max Wind SpeedKm/h'])

X_encoded.loc[:, 'dep_encod' : 'Revenue'] = StandardScaler().fit_transform(
                                                            X_encoded.loc[:, 'dep_encod' : 'Revenue'])

In [ ]:
X_encoded.drop(['Mean TemperatureC','Mean Humidity','Precipitationmm','CloudCover'], axis=1, inplace=True)
X_encoded.drop(['MeanDew PointC', 'Mean Sea Level PressurehPa'], axis=1, inplace=True)
X_encoded.drop(['ar_encod'], axis=1, inplace=True)

In [ ]:
y = X_encoded['log_PAX']
X_encoded.drop(['log_PAX'], axis=1, inplace=True)

In [ ]:
X_encoded.head()

In [ ]:
import xgboost as xgb
data_dmatrix = xgb.DMatrix(data=X_encoded,label=y)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

X_train, X_test, Y_train, Y_test = train_test_split(X_encoded, y, test_size=0.2)

In [ ]:
#Gridsearch
import xgboost as xgb
xgb1 = xgb.XGBRegressor()
parameters = {'nthread':[4],
              'objective':['reg:squarederror'],
              'learning_rate': [.2], 
              'max_depth': [7],
              'min_child_weight': [4],
              'subsample': [0.95],
              'colsample_bytree': [0.55],
              'n_estimators': [100,300,500,1000],
              'reg_lambda':[0.001],
              'gamma':[0]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train, Y_train)

print(xgb_grid.best_params_)

In [ ]:
parameters = {'nthread':4,
              'objective':'reg:squarederror',
              'learning_rate': .3, 
              'max_depth': 7,
              'min_child_weight': 4,
              'subsample': 0.95,
              'colsample_bytree': 0.55,
              'n_estimators': 1000,
              'reg_lambda':0.001,
              'gamma':0}

cv_results = xgb.cv(dtrain=data_dmatrix, params=parameters, nfold=3,
                    num_boost_round=300,early_stopping_rounds=5,metrics="rmse", as_pandas=True, seed=123)

In [ ]:
cv_results.head()

In [ ]:
print((cv_results["train-rmse-mean"]).tail(1))
print((cv_results["test-rmse-mean"]).tail(1))

In [ ]:
xg_reg = xgb.train(params=parameters, dtrain=data_dmatrix, num_boost_round=10)
xgb.plot_importance(xg_reg, max_num_features=40)
plt.rcParams['figure.figsize'] = [20, 20]
plt.show()

In [ ]:
import lightgbm as lgb 

In [ ]:
from lightgbm import LGBMRegressor
lightgbm = LGBMRegressor(task= 'train',
          boosting_type= 'gbdt',
          objective= 'regression',
          metric= {'l2','auc'},
          num_leaves= 300,
          learning_rate= 0.32,
          feature_fraction= 0.9,
          bagging_fraction= .9,
          bagging_freq= 70,
          verbose= 100)

In [ ]:
cv_results = lgb.cv(dtrain=data_dmatrix, params=parameters, nfold=3,
                    num_boost_round=300,early_stopping_rounds=5,metrics="rmse", as_pandas=True, seed=123)